In [1]:
import numpy as np


In [2]:
def read_glove_vecs(glove_file):
    with open(glove_file, 'r') as f:
        words = set()
        word_to_vec_map = {}
        
        for line in f:
            line = line.strip().split()
            curr_word = line[0]
            words.add(curr_word)
            word_to_vec_map[curr_word] = np.array(line[1:], dtype=np.float64)
            
    return words, word_to_vec_map


In [13]:
words, word_to_vec_map = read_glove_vecs('datasets/glove.6B.50d.txt')

In [14]:
def cosine_similarity(u, v):
    dot = np.dot(u, v)
    norm_u = np.linalg.norm(u)
    norm_v = np.linalg.norm(v)
    cosine_similarity = dot / (norm_u * norm_v)
    
    return cosine_similarity

In [17]:
father = word_to_vec_map["father"]
mother = word_to_vec_map["mother"]
ball = word_to_vec_map["ball"]
crocodile = word_to_vec_map["crocodile"]
france = word_to_vec_map["france"]
italy = word_to_vec_map["italy"]
paris = word_to_vec_map["paris"]
rome = word_to_vec_map["rome"]
jesus = word_to_vec_map["jesus"]
christ = word_to_vec_map["allah"]

print("cosine_similarity(father, mother) = ", cosine_similarity(father, mother))
print("cosine_similarity(ball, crocodile) = ",cosine_similarity(ball, crocodile))
print("cosine_similarity(france - paris, rome - italy) = ",cosine_similarity(france - paris, rome - italy))
print("cosine_similarity(jesus, christ) = ",cosine_similarity(jesus, christ))

cosine_similarity(father, mother) =  0.8909038442893618
cosine_similarity(ball, crocodile) =  0.2743924626137943
cosine_similarity(france - paris, rome - italy) =  -0.6751479308174203
cosine_similarity(jesus, christ) =  0.5743422355063569


In [18]:
def complete_analogy(word_a, word_b, word_c, word_to_vec_map): 
    word_a, word_b, word_c = word_a.lower(), word_b.lower(), word_c.lower()
    
    e_a, e_b, e_c = word_to_vec_map[word_a], word_to_vec_map[word_b], word_to_vec_map[word_c]
    
    words = word_to_vec_map.keys()
    max_cosine_sim = -100
    best_word = None
    
    for w in words:
        if w in [word_a, word_b, word_c]:
            continue
        
        cosine_sim = cosine_similarity(e_b - e_a, word_to_vec_map[w] - e_c)
        
        if cosine_sim > max_cosine_sim:
            max_cosine_sim = cosine_sim
            best_word = w
            
    return best_word

In [19]:
triads_to_try = [('italy', 'italian', 'spain'), ('india', 'delhi', 'japan'), ('man', 'woman', 'boy'), ('small', 'smaller', 'large')]
for triad in triads_to_try:
    print ('{} -> {} :: {} -> {}'.format( *triad, complete_analogy(*triad,word_to_vec_map)))

italy -> italian :: spain -> spanish
india -> delhi :: japan -> tokyo
man -> woman :: boy -> girl
small -> smaller :: large -> larger


In [24]:
g = word_to_vec_map['woman'] - word_to_vec_map['man']
print(g)

print("List of names and their similarities with constructed vector:")
name_list = ['john', 'marie', 'sophie', 'ronaldo', 'priya', 'rahul', 'danielle', 'reza', 'katy', 'yasmin']
for w in name_list:
    print(w, cosine_similarity(word_to_vec_map[w], g))



[-0.087144    0.2182     -0.40986    -0.03922    -0.1032      0.94165
 -0.06042     0.32988     0.46144    -0.35962     0.31102    -0.86824
  0.96006     0.01073     0.24337     0.08193    -1.02722    -0.21122
  0.695044   -0.00222     0.29106     0.5053     -0.099454    0.40445
  0.30181     0.1355     -0.0606     -0.07131    -0.19245    -0.06115
 -0.3204      0.07165    -0.13337    -0.25068714 -0.14293    -0.224957
 -0.149       0.048882    0.12191    -0.27362    -0.165476   -0.20426
  0.54376    -0.271425   -0.10245    -0.32108     0.2516     -0.33455
 -0.04371     0.01258   ]
List of names and their similarities with constructed vector:
john -0.23163356145973724
marie 0.31559793539607295
sophie 0.31868789859418784
ronaldo -0.3124479685032943
priya 0.17632041839009407
rahul -0.1691547103923172
danielle 0.24393299216283892
reza -0.0793042967219955
katy 0.2831068659572615
yasmin 0.23313857767928753


In [25]:
print("Other words and their similarities:")
word_list = ['lipstick', 'guns', 'science', 'arts', 'literature', 'warrior', 'doctor', 'tree', 'receptionist', 'technology', 'fashion', 'teacher', 'engineer', 'pilot', 'computer', 'singer']
for w in word_list:
    print(w, cosine_similarity(word_to_vec_map[w], g))

Other words and their similarities:
lipstick 0.2769191625638267
guns -0.1888485567898898
science -0.060829065409296994
arts 0.008189312385880351
literature 0.0647250443345993
warrior -0.20920164641125288
doctor 0.1189528941093504
tree -0.07089399175478091
receptionist 0.3307794175059374
technology -0.13193732447554293
fashion 0.03563894625772701
teacher 0.17920923431825667
engineer -0.08039280494524072
pilot 0.0010764498991917154
computer -0.10330358873850494
singer 0.18500518136496288


In [26]:
def neutralize(word, g, word_to_vec_map): 
    e = word_to_vec_map[word]
    e_biascomponent = (np.dot(e, g) / np.linalg.norm(g)**2) * g
    e_debiased = e - e_biascomponent
    
    return e_debiased

In [27]:
e = "receptionist"
print("cosine similarity between " + e + " and g, before neutralizing: ", cosine_similarity(word_to_vec_map["receptionist"], g))

e_debiased = neutralize("receptionist", g, word_to_vec_map)
print("cosine similarity between " + e + " and g, after neutralizing: ", cosine_similarity(e_debiased, g))

cosine similarity between receptionist and g, before neutralizing:  0.3307794175059374
cosine similarity between receptionist and g, after neutralizing:  -5.2569290990191626e-17
